In [2]:
# supress statsmodels warnings
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
from tqdm import tqdm
from IPython.display import display
import pickle


_start_year = 2017
_end_year = 2017
start_year = pd.to_datetime(f"{_start_year}-01-01 00:00:00")
end_year = pd.to_datetime(f"{_end_year}-12-31 23:59:59")

N_TICKERS = 300
focal_ticker = 'AAPL'

asset_type = 'stock'
period = '1min'
timeframe = 'full'
adjustment = 'adjsplitdiv'

# check operating system
if os.name == 'nt':
    CSV_DEST_DIR = f"E:/frd-historical/data/{asset_type}/{period}/csv/"
else:
    CSV_DEST_DIR = f"/media/reggie/reg_ext/frd-historical/data/{asset_type}/{period}/csv/"
print(f"{len(os.listdir(CSV_DEST_DIR))} files found in {CSV_DEST_DIR}")

6114 files found in E:/frd-historical/data/stock/1min/csv/


In [3]:
def add_trading_hours(df):
    """
    Add indicator for trading hours
    Parameters
    ----------
    df: pd.DataFrame
        Must have index as datetime
    """
    trading_start = 930
    trading_end = 1600
    df['time'] = df.index.strftime('%H%M').astype(int)
    df['is_trading_hour'] = (df['time'] >= trading_start) & (df['time'] <= trading_end)
    return df

def preprocess_data(data, nsamples=10_000, tickers='all', trading_hours=True):
    # Ensure data is correctly formatted
    if tickers == 'all':
        tickers = list(data.keys())
    else:
        tickers = tickers
    for ticker in data:
        data[ticker] = data[ticker].asfreq('1T').fillna(method='ffill')
        if nsamples > -1:
            data[ticker] = data[ticker].iloc[:nsamples]
        if trading_hours:
            df = data[ticker].copy()
            df = add_trading_hours(df)
            data[ticker] = df[df['is_trading_hour']]
    return data

def calculate_lagged_returns(data: dict, lags: int=3):
    for ticker in data:
        data[ticker][f'{ticker}_return'] = data[ticker][f'{ticker}_close'].pct_change()
        for i in range(1, lags + 1):
            # Calculate the lagged returns
            data[ticker][f'{ticker}_lag_{i}_return'] = data[ticker][f'{ticker}_close'].shift(1).pct_change(periods=i)
    return data

In [4]:
dtype={'date': str, 'open': float, 'high': float, 'low': float, 'close': float, 'volume': float}
columns = list(dtype.keys())
data = {}
tickers = [f.split("_")[0] for f in os.listdir(CSV_DEST_DIR) if f.endswith(".txt")][:N_TICKERS]
for ticker in tqdm(tickers):
    csv_path = os.path.join(CSV_DEST_DIR, f"{ticker}_{timeframe}_{period}_{adjustment}.txt")
    print(f"Reading {ticker} data from file {csv_path}")
    _columns = columns.copy()
    _columns[1:] = [f"{ticker}_{col}" for col in _columns[1:]]
    df = pd.read_csv(csv_path, names=_columns)
    df['date'] = pd.to_datetime(df['date'])
    df = df[['date', f'{ticker}_close']]
    df.set_index("date", inplace=True)
    try:
        df = df[(df.index >= start_year) & (df.index <= end_year)]    
    except:
        print(f"No data found for {ticker} for period {start_year} to {end_year}")
        continue
    if len(df) > 0:
        data[ticker] = df
    else:
        print(f"No data found for {ticker} for period {start_year} to {end_year}")

  0%|          | 0/300 [00:00<?, ?it/s]

Reading AACG data from file E:/frd-historical/data/stock/1min/csv/AACG_full_1min_adjsplitdiv.txt


  0%|          | 1/300 [00:00<02:07,  2.34it/s]

Reading AACT data from file E:/frd-historical/data/stock/1min/csv/AACT_full_1min_adjsplitdiv.txt
No data found for AACT for period 2017-01-01 00:00:00 to 2017-12-31 23:59:59
Reading AADI data from file E:/frd-historical/data/stock/1min/csv/AADI_full_1min_adjsplitdiv.txt


  1%|          | 3/300 [00:01<01:56,  2.54it/s]

...

 99%|█████████▉| 298/300 [05:00<00:02,  1.04s/it]

No data found for AMRX for period 2017-01-01 00:00:00 to 2017-12-31 23:59:59
Reading AMR data from file E:/frd-historical/data/stock/1min/csv/AMR_full_1min_adjsplitdiv.txt


100%|█████████▉| 299/300 [05:00<00:00,  1.16it/s]

Reading AMSC data from file E:/frd-historical/data/stock/1min/csv/AMSC_full_1min_adjsplitdiv.txt


100%|██████████| 300/300 [05:02<00:00,  1.01s/it]


In [5]:
data = preprocess_data(data)
data = calculate_lagged_returns(data)
for ticker in data:
    print(ticker, data[ticker].shape)

AACG (2657, 7)
AADI (2657, 7)
AAL (2737, 7)
AAMC (2737, 7)
AAME (2737, 7)
AAOI (2737, 7)
AAON (2737, 7)
AAPL (2737, 7)
AAP (2737, 7)
AAT (2737, 7)
AAU (2735, 7)
AA (2737, 7)
ABBV (2737, 7)
ABCB (2737, 7)
ABEO (2737, 7)
ABEV (2737, 7)
ABIO (2737, 7)
ABM (2737, 7)
ABR (2737, 7)
ABTS (2729, 7)
ABT (2737, 7)
ABUS (2737, 7)
AB (2737, 7)
ACAD (2737, 7)
ACCO (2737, 7)
ACEL (2657, 7)
ACGL (2737, 7)
ACHC (2737, 7)
ACHV (2737, 7)
ACIC (2737, 7)
ACIU (2737, 7)
ACIW (2737, 7)
ACLS (2737, 7)
ACMR (2657, 7)
ACM (2737, 7)
ACNB (2737, 7)
ACNT (2737, 7)
ACN (2737, 7)
ACOR (2737, 7)
ACP (2737, 7)
ACRE (2737, 7)
ACRS (2737, 7)
ACST (2737, 7)
ACTG (2737, 7)
ACU (2737, 7)
ACV (2737, 7)
AC (2737, 7)
ADAP (2737, 7)
ADBE (2737, 7)
ADC (2737, 7)
ADD (2715, 7)
ADEA (2737, 7)
ADI (2737, 7)
ADMA (2717, 7)
ADM (2737, 7)
ADNT (2737, 7)
ADP (2737, 7)
ADSK (2737, 7)
ADTN (2737, 7)
ADUS (2737, 7)
ADVM (2737, 7)
ADVWW (2657, 7)
ADX (2737, 7)
AEE (2737, 7)
AEG (2737, 7)
AEHL (2698, 7)
AEHR (2736, 7)
AEIS (2737, 7)
AEL (

In [ ]:
# Merge feature dataframes -- returns and lagged returns for all stocks
x = data[focal_ticker].copy()
x.drop(columns=[f'{focal_ticker}_close', 'is_trading_hour', 'time'], inplace=True)
n = len(x)
print(n)
for ticker in data:
    if ticker != focal_ticker:
        y = data[ticker].copy()
        y.drop(columns=[f'{ticker}_close', 'is_trading_hour', 'time'], inplace=True)

        # optional: skip tickers with fewer than n samples
        if y.shape[0] < n:
            print(f"Skipping {ticker} with shape {y.shape}")
            continue
        
        print(f"Merging {focal_ticker} and {ticker} with shapes {x.shape} and {y.shape}")
        x = x.merge(y, left_index=True, right_index=True, how='left')
        print(x.shape)
return_features_df = x.copy()
return_features_df = return_features_df.iloc[4:,] # because of lag NAs
print(return_features_df.shape)
print(return_features_df.info())
print(return_features_df)

# examine missing
print(return_features_df.shape)
for col in return_features_df.columns:
    missing = return_features_df[col].isnull().sum()
    if missing > 0:
        print(f"{col}: {missing} missing")
        return_features_df.drop(columns=[col], inplace=True)
print(return_features_df.shape)

with open('return_features_df.pickle', 'wb') as f:
    pickle.dump(return_features_df, f)